In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 데이터 셋: 타이타닉 데이터셋 (출처: Kaggle.com)

In [5]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


* PassengerId: 승객 아이디
* Survived: 생존 여부, 1: 생존, 0: 사망
* Pclass: 방 등급
* Name: 성함
* Sex: 성별
* Age: 나이
* SibSp: 형제, 자매, 배우자 수
* Parch: 부모, 자식 수
* Ticket: 티켓번호
* Fare: 요금
* Cabin: 좌석번호
* Embarked: 탑승 항구

## 전처리: train / validation 세트 나누기

1. 먼저, feature 와 label을 정의합니다.
2. feature / label을 정의했으면, 적절한 비율로 train / validation set을 나눕니다.

In [6]:
# 전체 데이터중 학습에 사용할 데이터를 선택하자.
feature = [
    'Pclass', 'Sex', 'Age', 'Fare'
]

In [7]:
# 예측 해야할 값
label = [
    'Survived'
]

In [8]:
train[feature].head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [9]:
train[label].head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [10]:
# 전체 데이터수를 확인해 보자
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
from sklearn.model_selection import train_test_split # 데이터를 쪼개준다

* **test_size**: validation set에 할당할 비율 (20% -> 0.2)
* **shuffle**: 셔플 옵션 (기본 True)
* **random_state**: 랜덤 시드값

전체 데이터를 train data와 validation data의 비율을 8:2로 나눈다.

In [12]:
x_train, x_valid, y_train, y_valid = train_test_split(train[feature], train[label], test_size=0.2, shuffle=True, random_state=30)

In [13]:
x_train.shape, y_train.shape

((712, 4), (712, 1))

In [14]:
x_valid.shape, y_valid.shape

((179, 4), (179, 1))

## 전처리: 결측치

In [15]:
#전체 데이터수 891개와 비교해서 갯수가 부족한 항목을 확인하자
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


결측치를 확인하는 방법은 pandas의 **isnull()** 

그리고 합계를 구하는 sum()을 통해 한 눈에 확인할 수 있습니다.

In [17]:
train.isnull().sum() # 결측치의 갯수를 구함

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

개별 column의 결측치에 대하여 확인하는 방법은 다음과 같습니다.

In [18]:
train['Age'].isnull().sum()

177

[Impute 도큐먼트](https://scikit-learn.org/stable/modules/impute.html)

### 1. 수치형 (Numerical Column) 데이터에 대한 결측치 처리

In [19]:
# 수치형 데이터는 대체로 평균 값을 구한뒤, 평균 값으로 채워준다.

In [20]:
# 0으로 채웠을때의 통계값을 출력해보자
train['Age'].fillna(0).describe()

count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [21]:
# 평균 값으로 채웠을때의 통계값을 출력해보자
train['Age'].fillna(train['Age'].mean()).describe()

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

#### 사이킷런의 imputer를 활용하면 2개 이상의 column을 한 번에 처리할 수 있다.

In [22]:
from sklearn.impute import SimpleImputer

In [23]:
imputer = SimpleImputer(strategy='mean')

fit() 을 통해 결측치에 대한 학습을 진행합니다.

In [25]:
imputer.fit(train[['Age', 'Pclass']]) 

SimpleImputer()

transform() 은 실제 결측치에 대한 처리를 해주는 함수합니다.

---



In [26]:
result = imputer.transform(train[['Age', 'Pclass']])

In [27]:
result

array([[22.        ,  3.        ],
       [38.        ,  1.        ],
       [26.        ,  3.        ],
       ...,
       [29.69911765,  3.        ],
       [26.        ,  1.        ],
       [32.        ,  3.        ]])

In [28]:
#결과값은 numpy.ndarray로 저장된다.
type(result)

numpy.ndarray

In [29]:
train[['Age', 'Pclass']] = result

In [30]:
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [31]:
train[['Age', 'Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


#### fit_transform()은 fit()과 transform()을 한 번에 해주는 함수 입니다.

In [32]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [33]:
train[['Age', 'Pclass']].isnull().sum()

Age       177
Pclass      0
dtype: int64

In [34]:
imputer = SimpleImputer(strategy='median') # mean : 평균값 / median : 중간값

In [35]:
result = imputer.fit_transform(train[['Age', 'Pclass']])

In [36]:
train[['Age', 'Pclass']] = result

In [37]:
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [38]:
train[['Age', 'Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.361582,2.308642
std,13.019697,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,28.000000,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


### 2.  (Categorical Column) 데이터에 대한 결측치 처리
문자형 데이터의 결측치를 채워줄 경우는 가장 빈도수가 높은 것으로 채워준다.

In [39]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

1개의 column을 처리하는 경우

In [40]:
train['Embarked'].fillna('S')

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

Imputer를 사용하는 경우 : 2개 이상의 column을 처리할 때

In [45]:
imputer = SimpleImputer(strategy='most_frequent')

In [46]:
result = imputer.fit_transform(train[['Embarked', 'Cabin']])

In [43]:
train[['Embarked', 'Cabin']] = result

In [44]:
train[['Embarked', 'Cabin']].isnull().sum()

Embarked    0
Cabin       0
dtype: int64

## Label Encoding : 문자(categorical)를 수치(numerical)로 변환

학습을 위해서 모든 문자로된 **데이터는 수치로 변환**하여야 합니다.

In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [49]:
# 아래와 같이 함수를 만들어서 문자열을 수치로 변환할 수 있다
def convert(data):
    if data == 'male':
        return 1
    elif data == 'female':
        return 0

In [50]:
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [51]:
train['Sex'].apply(convert)

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [52]:
# 사이킷런에서 LabelEncoder라는 함수를 사용할 수도 있다. 문자열을 숫자로 바꾸어준다 0, 1

In [53]:
from sklearn.preprocessing import LabelEncoder

In [54]:
le = LabelEncoder()

In [55]:
train['Sex_num'] = le.fit_transform(train['Sex'])

In [56]:
train['Sex_num'].value_counts()

1    577
0    314
Name: Sex_num, dtype: int64

In [57]:
le.classes_

array(['female', 'male'], dtype=object)

In [58]:
le.inverse_transform([0, 1, 1, 0, 0, 1, 1])

array(['female', 'male', 'male', 'female', 'female', 'male', 'male'],
      dtype=object)

fit_transform <---> inverse_transform

NaN 값이 포함되어 있다면, `LabelEncoder`가 정상 동작하지 않습니다.

In [59]:
le.fit_transform(train['Embarked'])

array([2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2,
       1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0,
       1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 2, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1,
       2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0,
       2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1,

In [60]:
train['Embarked'] = train['Embarked'].fillna('S')

In [61]:
le.fit_transform(train['Embarked'])

array([2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2,
       1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0,
       1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 2, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1,
       2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0,
       2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1,

## 원 핫 인코딩 (One Hot Encoding)

In [63]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [64]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


`Embarked`를 살펴봅시다

In [65]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [66]:
train['Embarked'] = train['Embarked'].fillna('S')

In [67]:
train['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [68]:
train['Embarked_num'] = LabelEncoder().fit_transform(train['Embarked'])

In [69]:
train['Embarked_num'].value_counts()

2    646
0    168
1     77
Name: Embarked_num, dtype: int64

Embarked는 탑승 항구의 이니셜을 나타냈습니다. 

그런데, 우리는 이전에 배운 내용처럼 `LabelEncoder`를 통해서 수치형으로 변환해주었습니다.

하지만, 이대로 데이터를 기계학습을 시키면, 기계는 데이터 안에서 관계를 학습합니다.

즉, 'S' = 2, 'Q' = 1 이라고 되어 있는데, `Q` + `Q` = `S` 가 된다 라고 학습해버린다는 것이죠.

그렇기 때문에, 독립적인 데이터는 별도의 column으로 분리하고,
각각의 컬럼에 해당 값에만 **True** 나머지는 **False**를 갖습니다.
우리는 이것은 원 핫 인코딩 한다라고 합니다.

In [70]:
train['Embarked'][:6]

0    S
1    C
2    S
3    S
4    S
5    Q
Name: Embarked, dtype: object

In [71]:
train['Embarked_num'][:6]

0    2
1    0
2    2
3    2
4    2
5    1
Name: Embarked_num, dtype: int64

In [72]:
pd.get_dummies(train['Embarked_num'][:6])

,0,1,2
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0


get_dummies --> 원핫인코딩으로 변환

In [76]:
one_hot = pd.get_dummies(train['Embarked_num'][:6])

In [77]:
one_hot.columns = ['C','Q','S']

In [78]:
one_hot

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0


위와 같이 column 을 분리시켜 **카테고리형 -> 수치형으로 변환**하면서 생기는 `수치형 값`의 관계를 끊어주어서 독립적인 형태로 바꾸어 줍니다.

**원핫인코딩**은 카테고리 (계절, 항구, 성별, 종류, 한식/일식/중식...)의 특성을 가지는 column에 대해서 적용 합니다.

## 전처리: Normalize (정규화)

부동산 가격을 예측하기 위해 상관관계가 있는 특성들을 도출한다고 하면,
집의 평수, 아파트의 나이, 지하철 역과의 거리, 대형마트와의 거리 등이
관련이 있을 것입니다. 

집의 평수는 10평부터 100평까지 다양할 것이고, 아파트의 나이는 1년에서 20년, 
지하철 역과의 거리는 0.1km부터 1km, 대형마트와의 거리도 1km에서 20km까지 다양할 것입니다. 
문제는 '평수', '년', 'km' 등 각 특성의 단위도 다르고, 값의 범위도 꽤 차이가 있다는 것입니다. 
우선 단위가 다르면 직접적인 비교가 불가능합니다. 사람의 키와 사람의 몸무게를 직접적으로 
비교할 수 없는 것처럼 말이죠.

따라서 특성들의 단위를 무시할 수 있도록, 또한 특성들의 값의 범위를 비슷하게 만들어줄 필요가 있습니다. 
그것이 바로 정규화 또는 표준화가 해주는 일입니다. 정규화와 표준화가 해주는 것을 특성 스케일링(feature scaling)
또는 데이터 스케일링(data scaling)이라고 부릅니다. 

참고 사이트 : https://bskyvision.com/849

column 간에 다른 **min**, **max** 값을 가지는 경우, 정규화를 통해 최소치/ 최대값의 척도를 맞추어 주는 것

* 네이버 영화평점 (0점 ~ 10점): [2, 4, 6, 8, 10]
* 넷플릭스 영화평점 (0점 ~ 5점): [1, 2, 3, 4, 5]

In [79]:
movie = {'naver': [2, 4, 6, 8, 10], 
         'netflix': [1, 2, 3, 4, 5]
         }

In [80]:
movie = pd.DataFrame(data=movie)
movie

,naver,netflix
0,2,1
1,4,2
2,6,3
3,8,4
4,10,5


In [81]:
from sklearn.preprocessing import MinMaxScaler

In [82]:
min_max_scaler = MinMaxScaler()

In [83]:
min_max_movie = min_max_scaler.fit_transform(movie)

In [84]:
pd.DataFrame(min_max_movie, columns=['naver', 'netflix'])

,naver,netflix
0,0.00,0.00
1,0.25,0.25
2,0.50,0.50
3,0.75,0.75
4,1.00,1.00


## 표준화 (Standard Scaling)

**평균이 0과 표준편차가 1**이 되도록 변환

In [85]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler()

샘플데이터를 생성합니다.

In [86]:
x = np.arange(10) # 0~9
# outlier 추가
x[9] = 1000

In [87]:
x.mean(), x.std() 

(103.6, 298.8100399919654)

In [88]:
scaled = standard_scaler.fit_transform(x.reshape(-1, 1))# reshape로 1차원 --> 2차원으로 변경

In [89]:
x.mean(), x.std()

(103.6, 298.8100399919654)

In [90]:
scaled.mean(), scaled.std()

(4.4408920985006264e-17, 1.0)

In [91]:
round(scaled.mean(), 2), scaled.std()

(0.0, 1.0)

노멀라이즈 - 데이터의 분포를 맞춰준다

표준화 - 서로 이상치에 대한것이 발견됬을 때 이상치를 최소화 하기 위해 